# Extrac Features

#เเก้ PATH ตรงนี้

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from skimage.feature import hog
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import cv2, glob, random, math, numpy as np, dlib, itertools
import os
import random
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model

image_types = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/Colab Notebooks/shape_predictor_68_face_landmarks.dat") 

def list_images(basePath, contains=None):
    return list_files(basePath, validExts=image_types, contains=contains)


def list_files(basePath, validExts=None, contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if validExts is None or ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename)
                yield imagePath

imagePaths = list(list_images('/content/drive/MyDrive/Colab Notebooks/images2'))

GET features and labes

In [ ]:
def colortogray(im):
    image = cv2.imread(im)
    imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return imgray

def feat_lab_HOG(imagePaths):

    features = []
    labels = []

    for imagePath in imagePaths:
        im = colortogray(imagePath)
        
        fd1 =  hog(im, orientations=7, pixels_per_cell=(8, 8),cells_per_block=(4, 4),block_norm= 'L2-Hys' ,transform_sqrt = False)

        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)
        features.append(fd1)

    features = np.array(features)
    labels = np.array(labels)
    return features,labels

def feat_lab_LAND(imagePaths):
      features = []
      labels = []
      for imagePath in imagePaths:
        im = colortogray(imagePath)
        clahe_image = clahe.apply(im)
        landmarks_vectorised = get_landmarks(clahe_image)
        if landmarks_vectorised == "error":
          pass
        else:
          label = imagePath.split(os.path.sep)[-2]
          labels.append(label)
          features.append(landmarks_vectorised)
      features = np.array(features)
      labels = np.array(labels)
      return features,labels

def feat_lab_LBP(imagePaths):
      features = []
      labels = []
      for imagePath in imagePaths:
        im = colortogray(imagePath)
        lbp_featured = lbp_feature(im)
        if lbp_featured == "error":
          pass
        else:
          label = imagePath.split(os.path.sep)[-2]
          labels.append(label)
          features.append(lbp_featured)
      features = np.array(features)
      labels = np.array(labels)
      return features,labels

def feat_lab_HLAC(imagePaths):
      features = []
      labels = []
      for imagePath in imagePaths:
        im = colortogray(imagePath)
        _, img = cv2.threshold(im, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        hlac_featured = calc_hlac_dev(img)
        if hlac_featured == "error":
          pass
        else:
          label = imagePath.split(os.path.sep)[-2]
          labels.append(label)
          features.append(hlac_featured)
      features = np.array(features)
      labels = np.array(labels)
      return features,labels

def feat_lab_GABOR(imagePaths):
      features = []
      labels = []
      for imagePath in imagePaths:
        im = colortogray(imagePath)
        gabor_featured = get_gabor(im)
        if gabor_featured == "error":
          pass
        else:
          label = imagePath.split(os.path.sep)[-2]
          labels.append(label)
          features.append(gabor_featured)
      features = np.array(features)
      labels = np.array(labels)
      return features,labels

In [ ]:
def get_landmarks(image):
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
            
        xmean = np.mean(xlist) #Get the mean of both axes to determine centre of gravity
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist] #get distance between each point and the central point in both axes
        ycentral = [(y-ymean) for y in ylist]

        if xlist[26] == xlist[29]: #If x-coordinates of the set are the same, the angle is 0, catch to prevent 'divide by 0' error in function
            anglenose = 0
        else:
            anglenose = int(math.atan((ylist[26]-ylist[29])/(xlist[26]-xlist[29]))*180/math.pi)

        if anglenose < 0:
            anglenose += 90
        else:
            anglenose -= 90

        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(x)
            landmarks_vectorised.append(y)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            anglerelative = (math.atan((z-ymean)/(w-xmean))*180/math.pi) - anglenose
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append(anglerelative)

        if len(detections) < 1: 
          landmarks_vectorised = "error"
    return landmarks_vectorised

LBP

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    '''

     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    

    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val    

def show_output(output_list):
    output_list_len = len(output_list)
    figure = plt.figure()
    for i in range(output_list_len):
        current_dict = output_list[i]
        current_img = current_dict["img"]
        current_xlabel = current_dict["xlabel"]
        current_ylabel = current_dict["ylabel"]
        current_xtick = current_dict["xtick"]
        current_ytick = current_dict["ytick"]
        current_title = current_dict["title"]
        current_type = current_dict["type"]
        current_plot = figure.add_subplot(1, output_list_len, i+1)
        if current_type == "gray":
            current_plot.imshow(current_img, cmap = plt.get_cmap('gray'))
            current_plot.set_title(current_title)
            current_plot.set_xticks(current_xtick)
            current_plot.set_yticks(current_ytick)
            current_plot.set_xlabel(current_xlabel)
            current_plot.set_ylabel(current_ylabel)
        elif current_type == "histogram":
            current_plot.plot(current_img, color = "black")
            current_plot.set_xlim([0,260])
            current_plot.set_title(current_title)
            current_plot.set_xlabel(current_xlabel)
            current_plot.set_ylabel(current_ylabel)            
            ytick_list = [int(i) for i in current_plot.get_yticks()]
            current_plot.set_yticklabels(ytick_list,rotation = 90)

    plt.show()
    
def lbp_feature(image):
    img_bgr = image
    
    height, width = img_bgr.shape
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_lbp = np.zeros((height, width,3), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
             img_lbp[i, j] = lbp_calculated_pixel(image, i, j)
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    return hist_lbp

HLAC

In [ ]:
import numpy as np
import pandas as pd
import cv2

def calc_chlac_dev(div_data, masks, mask_n):
    chlacs = np.array([])
    for m, mn in zip(masks, mask_n):
        chlacs = np.append(
            chlacs, (np.sum(np.sum(np.logical_and(m, div_data), axis=1) == mn))
        )
    return chlacs


# old version, too slow
def calc_chlac(div_data, masks, mask_n):
    chlacs = []
    for m, mn in zip(masks, mask_n):
        logic = np.logical_and(m, div_data)
        logic_sum = np.sum(logic, axis=1)
        chlacs.append(np.sum(logic_sum == int(mn)))
    return np.array(chlacs)


def split2boxel(data):
    x, y = data[0].shape
    d = 3
    li = []
    # 最外周は外してnp.whereを実行
    # np.whereは切り出したndarrayに対して条件に一致するインデックスを返す
    idxs_x, idxs_y = np.where(data[1, 1:-1, 1:-1] == 255)
    # i, jは切り出すボクセルの左上隅のインデックスを表す
    for i, j in zip(idxs_x, idxs_y):
        li.append(data[:, i: i + d, j: j + d])
    div_data = np.array(li).reshape([len(li), 27]).astype(np.int32)

    return div_data


# old version, too slow
def split2boxel_(data):
    x, y = data[0].shape
    d = 3
    li = []
    for i in range(x - 2):
        for j in range(y - 2):
            if data[1, i + 1, j + 1] != 0:
                li.append(data[:, i: i + d, j: j + d])
    div_data_ = np.array(li).reshape([len(li), 27]).astype(np.int32)

    return div_data_


def prepare_masks_chlac(mask_filepath):
    data = pd.read_csv(mask_filepath, header=None, index_col=0)
    N_mask = data.shape[0]

    masks = []
    for j in range(N_mask):
        mask = np.zeros([3, 3, 3])  # voxel
        _ = []
        for i in range(3):
            if data.iloc[j, i] == "x":
                _.append([])
            elif data.iloc[j, i] != "x":
                if len(data.iloc[j, i]) == 1:
                    _.append([data.iloc[j, i]])
                else:
                    _.append(data.iloc[j, i].split(","))

        for i in range(3):
            if len(_[i]) == 0:
                continue
            for m in _[i]:
                if m == "a":
                    mask[0, i, 0] = 1
                elif m == "b":
                    mask[1, i, 0] = 1
                elif m == "c":
                    mask[2, i, 0] = 1
                elif m == "d":
                    mask[0, i, 1] = 1
                elif m == "e":
                    mask[1, i, 1] = 1
                elif m == "f":
                    mask[2, i, 1] = 1
                elif m == "g":
                    mask[0, i, 2] = 1
                elif m == "h":
                    mask[1, i, 2] = 1
                elif m == "i":
                    mask[2, i, 2] = 1
        mask = mask.reshape(mask.shape[0] * mask.shape[1] * mask.shape[2])
        masks.append(mask)
    masks = np.array(masks)
    mask_n = np.array(masks).sum(axis=1)

    return masks, mask_n


def read_testdata():
    imgs = 255 * np.array(
        [
            [
                [1, 0, 1, 1, 0, 1],
                [1, 1, 0, 0, 1, 1],
                [0, 0, 1, 0, 0, 0],
                [1, 1, 1, 1, 1, 0],
                [1, 1, 1, 1, 1, 1],
                [0, 0, 0, 1, 0, 0],
            ],
            [
                [1, 1, 1, 0, 0, 1],
                [1, 1, 1, 0, 0, 1],
                [0, 1, 1, 0, 1, 0],
                [1, 1, 0, 1, 1, 0],
                [1, 1, 1, 0, 0, 0],
                [0, 1, 0, 0, 0, 0],
            ],
            [
                [1, 1, 0, 0, 1, 1],
                [0, 0, 1, 1, 0, 1],
                [0, 1, 0, 1, 0, 1],
                [1, 1, 1, 0, 0, 0],
                [1, 0, 0, 1, 0, 0],
                [0, 1, 1, 0, 1, 1],
            ],
        ]
    )

    return imgs


def calc_hlac_dev(img, dim=3):
    masks_origin = [
        "000010000",
        "000011000",
        "001010000",
        "010010000",
        "100010000",
        "000111000",
        "001010100",
        "010010010",
        "100010001",
        "001110000",
        "010010100",
        "100010010",
        "000110001",
        "000011100",
        "001010010",
        "010010001",
        "100011000",
        "010110000",
        "100010100",
        "000110010",
        "000010101",
        "000011010",
        "001010001",
        "010011000",
        "101010000",
    ]
    masks = []
    masks_n = []

    # make masks
    for mask_bin in masks_origin:
        m = []
        s = 0
        for ch in mask_bin:
            m.append(int(ch))
            if int(ch) == 1:
                s += 1   
                            
        masks.append(np.array(m).reshape((3, 3)))
        masks_n.append(s)

    height, width = img.shape

    a = patchify(img, (3, 3))
    a = a.reshape(
        (height - 2) * (width - 2), 3, 3
    )  # (x-2, y-2, 3) -> ((x-2)*(y-2), 3, 3)

    ret = []
    for mask, n in zip(masks, masks_n):
        res = np.logical_and(mask, a)
        logic = np.sum(np.sum(res, axis=2), axis=1)
        ret.append(np.sum(logic == n))

    return ret


def patchify(img, patch_shape):
    img = np.ascontiguousarray(img)  # won't make a copy if not needed
    X, Y = img.shape
    x, y = patch_shape
    shape = ((X - x + 1), (Y - y + 1), x, y)  # number of patches, patch_shape
    # The right strides can be thought by:
    # 1) Thinking of `img` as a chunk of memory in C order
    # 2) Asking how many items through that chunk of memory are needed when indices
    #    i,j,k,l are incremented by one
    strides = img.itemsize * np.array([Y, 1, Y, 1])
    return np.lib.stride_tricks.as_strided(img, shape=shape, strides=strides)

GABOR

In [ ]:
import numpy as np
import cv2
import pandas as pd
from scipy import misc
import scipy.io as sio
from skimage.color import rgb2gray
def get_gabor(image):
    
    input_img = image
    if input_img.ndim == 3 and input_img.shape[-1] == 3:
        img = cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
    elif input_img.ndim == 2:
        img = input_img
    else:
        raise Exception("The module works only with grayscale and RGB images!")
    pixel_values = image.reshape(-1)
    Pixel_Value = pixel_values   #Pixel value itself as a feature
    #df['Image_Name'] = image   #Capture image name as we read multiple images
    num = 1  #To count numbers up in order to give Gabor features a lable in the data frame
    kernels = []  #Create empty list to hold all kernels that we will generate in a loop
    for theta in range(8):   #Define number of thetas. Here only 2 theta values 0 and 1/4 . pi 
        theta = theta / 4. * np.pi
        for sigma in (1, 3,):  #Sigma with values of 1 and 3
            for lamda in np.arange(0, np.pi, np.pi / 4):   #Range of wavelengths
                for gamma in (0.05, 0.5):   #Gamma values of 0.05 and 0.5
                           
                    gabor_label = 'Gabor' + str(num)  #Label Gabor columns as Gabor1, Gabor2, etc.
#                   print(gabor_label)
                    ksize=9  #Try 15 for hidden image. Or 9 for others
                    phi = 0  #0.8 for hidden image. Otherwise leave it to 0
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)    
                    kernels.append(kernel)
                    #Now filter the image and add values to a new column 
                    fimg = cv2.filter2D(image, cv2.CV_8UC3, kernel)                
                    filtered_img = fimg.reshape(-1)
                
                    gabor_label = filtered_img  #Labels columns as Gabor1, Gabor2, etc.
                    #print(gabor_label, ': theta=', theta, ': sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
                    num += 1  #Increment for gabor column label
    return gabor_label

In [ ]:
features_HOG,labels_HOG = feat_lab_HOG(imagePaths)
features_LAND,labels_LAND = feat_lab_LAND(imagePaths)
features_LBP,labels_LBP = feat_lab_LBP(imagePaths)
features_HLAC,labels_HLAC = feat_lab_HLAC(imagePaths)
features_GABOR,labels_GABOR = feat_lab_GABOR(imagePaths) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [ ]:
print("LAND: ",features_LAND)
print("HOG: ",features_HOG)
print("LBP: ",features_LBP)
print("HLAC: ",features_HLAC)
print("GABOR",features_GABOR)

print("LAND: ",features_LAND.shape)
print("HOG: ",features_HOG.shape)
print("LBP: ",features_LBP.shape)
print("HLAC: ",features_HLAC.shape)
print("GABOR",features_GABOR.shape)

LAND:  [[-46.01492537 -13.07462687  47.83637972 ...  25.92537313  27.13602773
  -74.82081183]
 [-44.71641791 -13.97014925  46.84787189 ...  27.02985075  28.72318938
  -74.22801571]
 [-47.31343284 -17.02985075  50.28495544 ...  23.97014925  24.35515876
  -77.79879082]
 ...
 [-44.1641791   -8.8358209   45.03938773 ...  30.1641791   31.00327665
  -84.6394461 ]
 [-63.01492537 -19.43283582  65.94327811 ...  26.56716418  27.47768896
  -70.2089151 ]
 [-50.3880597   -5.80597015  50.72145355 ...  29.19402985  30.11436211
  -83.79846084]]
HOG:  [[0.31295983 0.00451276 0.0014583  ... 0.0622666  0.08588023 0.29132184]
 [0.21357372 0.07510764 0.0083327  ... 0.00327694 0.06911241 0.3196029 ]
 [0.02762948 0.01551961 0.         ... 0.         0.00113458 0.0020454 ]
 ...
 [0.09654229 0.22587123 0.22587123 ... 0.01369306 0.04037822 0.18140746]
 [0.0218669  0.04179065 0.04890871 ... 0.02255598 0.02986664 0.01657264]
 [0.09487885 0.         0.         ... 0.03106328 0.17537065 0.28426754]]
LBP:  [[[117.]


# SAVE FILE ด้วย

In [ ]:
np.save("features_HOG", features_HOG)
np.save("features_HLAC", features_HLAC)
np.save("features_LAND", features_LAND)
np.save("features_LBP", features_LBP)
np.save("features_GABOR", features_GABOR)

np.save("labels_HOG", labels_HOG)
np.save("labels_HLAC", labels_HLAC)
np.save("labels_LAND", labels_LAND)
np.save("labels_LBP", labels_LBP)
np.save("labels_GABOR", labels_GABOR)